In [1]:
from _plotly_future_ import v4_subplots

In [2]:
import pandas as pd
import numpy as np

In [3]:
from collections import Counter

In [4]:
from utils import EvalMetric

Using TensorFlow backend.


In [5]:
import matplotlib.pyplot as plt 
# import seaborn as sns
import pandas as pd
# sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')

In [6]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [7]:
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_10_100_100.2019_09_12_1.sample100.pkl'
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_50_100_100.2019_09_12_1.sample100.pkl'
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_50_100_100.2019_09_17_1.sample100.pkl'
result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_50_100_100.2019_09_25_1.sample100.pkl'
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_50_100_100.2019_09_27_1.sample100.pkl'
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_50_100_100.2019_10_09_3.sample100.pkl'
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_50_100_100.2019_10_14_2.sample100.pkl'
# result_filenmae = 'result/prediction.rand_seqs_var_len_sample_ensemble_100_200_100.2019_09_25_1.sample100.pkl'

In [8]:
df = pd.read_pickle(result_filenmae)


In [9]:
df.head()

,one_idx,pred_idx,seq
0,"[([4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, ...","[([33, 34, 35, 36, 61, 62, 63, 64, 65, 66, 67,...",CUAGAGGGUUCGGCAAGGUGUGGAGCCCGUAGGGAGGAACUCAAUG...
1,"[([2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 1...","[([2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 1...",GGGAUGGAGUUCAGUCACCCUCAGGGGACUACUUAACUUGUAGCCC...
2,"[([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 33,...","[([16, 17, 18, 51, 52, 53], [87, 86, 85, 61, 6...",CAGGCAGGAUUAAGUGGGCCAAGUUGAUUCCUAUACAUCCGUAGUU...
3,"[([5, 6, 9, 10, 11, 26, 27, 28, 29, 30, 31, 32...","[([9, 10, 11, 36, 37, 38], [20, 19, 18, 45, 44...",AUAGAUGAAGGCACCAUAGCCAUAAAUUAUAUUAGUUGCCAGUGCA...
4,"[([1, 2, 3, 4, 9, 10, 11, 16, 17, 18, 47, 48, ...","[([10, 11, 13, 14, 15, 16, 17], [35, 34, 32, 3...",UCCAUCAGACCGAUCAUGAAUUUUGCGACGUGGACGUGGAUCGACU...


In [10]:
# convert to tuple so we can use set()
def _to_tuple(idxes):
    x = []
    for _a in idxes:
        a = []
        for i, j in zip(_a[0], _a[1]):
            a.append((i, j))
        if len(a) > 0:
            a = tuple(a)
            x.append(a)
    return x

In [11]:
def _to_mat(x, l):
    # x is tuple of tuple (2 elements)
    # convert to list of 2 lists
    y = [[a[0] for a in x], [a[1] for a in x]]
    z = np.zeros((l, l))
    z[y] = 1
    return z

In [12]:
n_no_struct = 0
df_metric = []

for _, row in df.iterrows():
    seq = row['seq']
    rnafold_struct_count = Counter(_to_tuple(row['one_idx']))
    model_struct_count = Counter(_to_tuple(row['pred_idx']))
#     # only keep structures that occurs more than 2 times (2% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=2}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=2}
#     # only keep structures that occurs more than 5 times (5% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=5}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=5}
#     # only keep structures that occurs more than 10 times (10% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=10}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=10}
    
    # skip the case if RNAfold outputs no structure
    if len(rnafold_struct_count) == 0:
        n_no_struct += 1
        continue
    
    # for each structure from RNAfold
    # evaluate against all structures from the model, pick the closest one, record performance
#     result = []
    for rnafold_struct, rnafold_count in rnafold_struct_count.iteritems():
        r_s = _to_mat(rnafold_struct, len(seq))
        sensitivity = 0
        ppv = 0
        f_measure = 0
        best_m_s = None
        for model_struct in model_struct_count.keys():
            m_s = _to_mat(model_struct, len(seq))
            _sensitivity = EvalMetric.sensitivity(m_s, r_s)
            _ppv = EvalMetric.ppv(m_s, r_s)
            _f_measure = EvalMetric.f_measure(_sensitivity, _ppv)
            if _f_measure > f_measure:
                sensitivity = _sensitivity + 0  # copy
                ppv = _ppv + 0
                f_measure = _f_measure + 0
                best_m_s = list(model_struct)
#         result.append((sensitivity, ppv, f_measure))
        df_metric.append({
            'seq': seq,
            'rnafold_idx': list(rnafold_struct),
            'closest_model_idx': best_m_s,
            'sensitivity': sensitivity,
            'ppv': ppv,
            'f_measure': f_measure,
            'rnafold_count': rnafold_count,
        })
    
#     print rnafold_struct_count
#     print model_struct_count
#     print result
#     print ''
    
    
print n_no_struct
df_metric = pd.DataFrame(df_metric)


/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

utils.py:325: RuntimeWarning:

invalid value encountered in double_scalars



0


In [13]:
df_metric.head()

,closest_model_idx,f_measure,ppv,rnafold_count,rnafold_idx,sensitivity,seq
0,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.866667,0.928571,3,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.812500,CUAGAGGGUUCGGCAAGGUGUGGAGCCCGUAGGGAGGAACUCAAUG...
1,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.923077,0.960000,1,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.888889,CUAGAGGGUUCGGCAAGGUGUGGAGCCCGUAGGGAGGAACUCAAUG...
2,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.896552,0.928571,1,"[(4, 90), (5, 89), (6, 88), (9, 85), (10, 84),...",0.866667,CUAGAGGGUUCGGCAAGGUGUGGAGCCCGUAGGGAGGAACUCAAUG...
3,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.857143,0.954545,1,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.777778,CUAGAGGGUUCGGCAAGGUGUGGAGCCCGUAGGGAGGAACUCAAUG...
4,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.931034,0.964286,1,"[(4, 90), (5, 89), (6, 88), (8, 86), (9, 85), ...",0.900000,CUAGAGGGUUCGGCAAGGUGUGGAGCCCGUAGGGAGGAACUCAAUG...


In [24]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Sensitivity', 'PPV'])

for col_idx, metric_name in enumerate(['sensitivity', 'ppv']):

    col = col_idx + 1
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 1][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 1])),
                              marker_color='#ffffcc', showlegend=True if col_idx == 0 else False), 
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 2][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 2])),
                              marker_color='#c7e9b4', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 3][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 3])),
                              marker_color='#7fcdbb', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 4][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 4])),
                              marker_color='#41b6c4', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 5][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 5])),
                              marker_color='#1d91c0', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)]['sensitivity'], 
                              nbinsx=20, histnorm='probability', 
                               name='(5, 10] ({})'.format(len(df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)])),
                              marker_color='#225ea8', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] > 10][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='>10 ({})'.format(len(df_metric[df_metric['rnafold_count'] > 10])),
                              marker_color='#0c2c84', showlegend=True if col_idx == 0 else False), 
                 row=1, col=col)

fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
#     xaxis=dict(showgrid=False, zeroline=False,
#                tickfont=dict(size=18, color='#000'),
#                titlefont=dict(size=24, color='#000'),
#                showline=True, mirror=True),
#     yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
#                tickfont=dict(size=18, color='#000'),
#                titlefont=dict(size=24, color='#000'),
#                title='Percentage', showline=True, mirror=True),
    title=result_filenmae,
#     title=metric_name,
#     titlefont=dict(size=24, color='#000'),
#     legend=dict(orientation="h",
#                 font=dict(
# #             family='sans-serif',
#             size=24,
#             color='#000'
#         ),),
))
iplot(fig)

In [15]:
# for metric_name in ['sensitivity', 'ppv']:
#     fig = go.Figure()
#     # fig = df_metric[df_metric['rnafold_count'] > 5][['sensitivity', 'ppv']].iplot(kind='histogram', bins=20,
#     #                                                     histnorm='probability', opacity=1,
#     #                                                      barmode='group', asFigure=True)

#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 1][metric_name], 
#                               nbinsx=20, histnorm='probability', name='1'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 2][metric_name], 
#                               nbinsx=20, histnorm='probability', name='2'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 3][metric_name], 
#                               nbinsx=20, histnorm='probability', name='3'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 4][metric_name], 
#                               nbinsx=20, histnorm='probability', name='4'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 5][metric_name], 
#                               nbinsx=20, histnorm='probability', name='5'))
#     fig.add_trace(go.Histogram(x=df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)]['sensitivity'], 
#                               nbinsx=20, histnorm='probability', name='(5, 10]'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] > 10][metric_name], 
#                               nbinsx=20, histnorm='probability', name='>10'))

#     fig = fig.update(layout=dict(
#         paper_bgcolor='rgba(0,0,0,0)',
#         plot_bgcolor='rgba(0,0,0,0)',
#         xaxis=dict(showgrid=False, zeroline=False,
#                    tickfont=dict(size=18, color='#000'),
#                    titlefont=dict(size=24, color='#000'),
#                    showline=True, mirror=True),
#         yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
#                    tickfont=dict(size=18, color='#000'),
#                    titlefont=dict(size=24, color='#000'),
#                    title='Percentage', showline=True, mirror=True),
#     #     title='Performance on test dataset',
#         title=metric_name,
#         titlefont=dict(size=24, color='#000'),
#     #     legend=dict(orientation="h",
#     #                 font=dict(
#     # #             family='sans-serif',
#     #             size=24,
#     #             color='#000'
#     #         ),),
#     ))
#     iplot(fig)

In [16]:
# fig = df_metric[['sensitivity', 'ppv','f_measure']].iplot(kind='histogram', bins=20,
#                                                     histnorm='probability',
#                                                      barmode='group', asFigure=True)
fig = df_metric[['sensitivity', 'ppv']].iplot(kind='histogram', bins=20,
                                                    histnorm='probability', opacity=1,
                                                     barmode='group', asFigure=True)
fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=False, zeroline=False,
               tickfont=dict(size=18, color='#000'),
               titlefont=dict(size=24, color='#000'),
               showline=True, mirror=True),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
               tickfont=dict(size=18, color='#000'),
               titlefont=dict(size=24, color='#000'),
               title='Percentage', showline=True, mirror=True),
    title='Performance on test dataset',
    titlefont=dict(size=24, color='#000'),
    legend=dict(orientation="h",
                font=dict(
#             family='sans-serif',
            size=24,
            color='#000'
        ),),
))

In [17]:
iplot(fig)

In [18]:
# # uncomment to show plot in new window
# plot(fig)

In [19]:
len(df_metric)

6676

In [20]:
sum(df_metric['ppv']>0.8)/float(len(df_metric))

0.799131216297184

In [21]:
# # for R plot
# df_metric[['seq', 'sensitivity', 'ppv', 'f_measure']].to_csv('/Users/alicegao/data/tmp/perf_rand_ensemble_metric.csv', index=False)

In [22]:
# # scatter plot

# fig = df_metric[['sensitivity', 'ppv']].iplot(kind='scatter', x='sensitivity', y='ppv',
#                                                     mode='markers', size=4,
#                                               asFigure=True)
# fig = fig.update(layout=dict(
#     paper_bgcolor='rgba(0,0,0,0)',
#     plot_bgcolor='rgba(0,0,0,0)',
#     xaxis=dict(showgrid=False, title='Sensitivity', zeroline=False),
#     yaxis=dict(showgrid=False, title='PPV'),
#     title='Performance on test dataset',
# ))

In [23]:
# iplot(fig)